In [ ]:
# Write a program to predict prices of a stock using simple RNN

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,Dense
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/TESLA.csv")
df.head()


In [ ]:
df.info()

In [ ]:
# Converting Strings to Date format
df['Date'] = pd.to_datetime(df['Date'],dayfirst = True)
df.info()

In [ ]:
# When the dataset is a timeseries , the rows must be in chronological order
# for time based analysis
# we will sort the dataset
df.sort_values("Date",inplace = True)
df.head()

In [ ]:
# Select the output column which is to be predicted
# we will select the close column
data = df[["Close"]].values

In [ ]:
# Normalise the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# creating the sequences
def create_sequences(data, seq_length = 60):
  X,y = [],[]
  for i in range(seq_length,len(data)):
    X.append(data[i-seq_length:i,0])
    y.append(data[i,0])

  return np.array(X),np.array(y)

In [ ]:
# Calling the function and storing the values in X, y
sequence_length = 60
X, y = create_sequences(scaled_data, sequence_length)
X = X.reshape((X.shape[0],X.shape[1],1))

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, shuffle = False)

In [ ]:
# Building the model
model = Sequential([
    SimpleRNN(50, activation='tanh',input_shape=(X_train.shape[1],1)),
    Dense(1) # Output is a single value
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',loss = 'mse')

In [ ]:
# Print model summary
model.summary()

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data = (X_test, y_test))

In [ ]:
# Print and evaluate on Test set
predicted = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted.reshape(-1,1))
actual_prices = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
# Printing predicted prices and actual prices
print("Predicted Prices are:",predicted_prices)
print("Actual Price is:", actual_prices)